In [1]:
import sys
import os
import pandas as pd
import numpy as np
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.mllib.feature import StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark = SparkSession.builder.config("spark.executor.pyspark.memory", "6g") \
        .config("spark.executor.memory", "6g").config("spark.driver.memory", "6g") \
        .config("spark.driver.maxResultSize", "6g").getOrCreate()

In [3]:
select_cols = input()
if(int(select_cols)):
    df = spark.read.csv('hdfs://localhost:9000/user/sidlohia/data/datasetfinal.csv', sep=',', inferSchema=True, header=True).select("Duration","Dst_bytes","Flag","Hot","Num_compromised","Count","Serror_rate","Rerror_rate","Same_srv_rate","Diff_srv_rate","Srv_count","Srv_serror_rate","Srv_rerror_rate","Srv_diff_host_rate","Dst_host_count","Dst_host_srv_count","Dst_host_same_srv_rate","Dst_host_diff_srv_rate","Dst_host_same_src_port_rate","Dst_host_srv_diff_host_rate","Dst_host_serror_rate","Dst_host_srv_serror_rate","Dst_host_rerror_rate","class","class_index")
else:
    df = spark.read.csv('hdfs://localhost:9000/user/sidlohia/data/datasetfinal.csv', sep=',', inferSchema=True, header=True)

KeyboardInterrupt: Interrupted by user

In [ ]:
spark.sparkContext

In [ ]:
df.dtypes

In [ ]:
catcols = [x for (x, dataType) in df.dtypes if dataType == "string"]
numcols = [x for (x, dataType) in df.dtypes if ((dataType =="int") | (dataType == "double") & (x != "class_index"))]

In [ ]:
numcols

In [ ]:
from pyspark.ml.feature import StandardScaler, VectorAssembler 
vector_assembler = VectorAssembler(inputCols = numcols, outputCol = "features")
temptrain = vector_assembler.setHandleInvalid("skip").transform(df).select("class_index","features")

In [ ]:
# temptrain.select("class_index","features").show(5)

In [ ]:
df = StandardScaler(inputCol = "features", outputCol="scaledfeatures").fit(temptrain).transform(temptrain)

In [ ]:
# train, test = df.randomSplit([0.7, 0.3], seed = 2022)

In [ ]:
# train.select("scaledfeatures").show(5)

In [ ]:
select_classifier = input()

In [ ]:
if(int(select_classifier)):
    model = RandomForestClassifier(featuresCol = 'scaledfeatures', labelCol = 'class_index').fit(df)
else:
    model = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol = 'scaledfeatures', labelCol = 'class_index').fit(df)

In [4]:
test = spark.read.csv('/home/sidlohia/Majorproject/NSLKDD/KDDTrain+.csv')

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

In [ ]:
evaluator = MulticlassClassificationEvaluator( labelCol="class_index", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(predictions)
accuracy

In [ ]:
# rf.featureImportances

In [ ]:
if(int(select_classifier)):
    figure(figsize=(12, 10), dpi=100)
    plt.barh(numcols, model.featureImportances)

In [ ]:
# model.save('/home/sidlohia/rfmodel')

In [ ]:
# rf = RandomForestClassificationModel.load('hdfs://localhost:9000/user/sidlohia/data/rfmodel')

In [ ]:
# rf